In [1]:
import pandas as pd

In [5]:
data = pd.read_excel('./current_offers.xlsx')
data.head()

,NAME,WEB,CASH_BACK_HEIGHT,TRANCHE_STMT_COUNT,OFFER_TYPE,ADVERT_TEXT
0,ANEX Tour,https://www.anextour.com/,0,6.0,SPECIAL_CREDIT,На покупку по кредитной карте не будут начисля...
1,Гостиница Альфа,http://www.alfa-hotel.ru/,15,NaN,STANDARD,Гостиница Альфа 4* – разумный выбор для отдыха...
2,MirCli,https://mircli.ru/?utm_source=Tinkoff&utm_medi...,0,4.0,SPECIAL_CREDIT,На покупку по кредитной карте не будут начисля...
3,MIXIT,https://mixit.ru,10,NaN,STANDARD,MIXIT - лаборатория современной косметики. Это...
4,Tinkoff.Travel совместно с Booking.com,https://travel.tinkoff.ru/hotels/?utm_source=p...,10,NaN,STANDARD,Бронирование отелей через Тинькофф Путешествия...


In [4]:
web = pd.read_excel('./offers_with_categories.xlsx')
web.head()

,OFFER_ID,NAME,CASH_BACK_HEIGHT,TRANCHE_STMT_COUNT,OFFER_TYPE,ADVERT_TEXT,CATEGORY_NAME
0,5352,Tinkoff.Travel совместно с Rentalcars,0,NaN,STANDARD,Аренда автомобилей через Тинькофф Путешествия ...,Каникулы в Сочи
1,5352,Tinkoff.Travel совместно с Rentalcars,0,NaN,STANDARD,Аренда автомобилей через Тинькофф Путешествия ...,Путешествия
2,5354,Tinkoff.Travel совместно с Booking.com,10,NaN,STANDARD,Бронирование отелей через Тинькофф Путешествия...,Каникулы в Сочи
3,5354,Tinkoff.Travel совместно с Booking.com,10,NaN,STANDARD,Бронирование отелей через Тинькофф Путешествия...,Путешествия
4,5515,Gett,10,NaN,STANDARD,Gett — один из мировых лидеров заказа такси. С...,Для детей


In [95]:
def merge(cat):
    offer_with_cats = []
    for _, row in data.iterrows():
        cats = web[web['NAME'] == row['NAME']]
        cats = cats[cats['CATEGORY_NAME'] == cat]
        if len(cats) == 0:
            continue

        if len(cats) > 1:
            #sc = cats[cats['OFFER_TYPE'] == 'SPECIAL_CREDIT']
            cats = cats[cats['ADVERT_TEXT'] == row['ADVERT_TEXT']]
            cats = cats[cats['CASH_BACK_HEIGHT'] > 0]
            if len(cats) == 0:
                continue

        if len(cats) > 1:
            cats = cats[:1]

        if row['WEB'] != row['WEB']:
            continue

        assert len(cats) == 1, cats
        offer_with_cats.append(row)
        
    return pd.DataFrame(offer_with_cats)

In [96]:
sport = merge('Спорт')
len(sport)

29

In [102]:
sport.to_excel('sport.xls', index=False)

In [97]:
eda = merge('Еда и продукты')
len(eda)

36

In [103]:
eda.to_excel('food.xls', index=False)